# Run Simulations

## Import relevant libraries

In [1]:
import warnings
warnings.filterwarnings('ignore')

from configparser import ConfigParser

from typing import Callable

import gym
import gym_anytrading

from gym_anytrading.envs import CryptoEnvLogBLSH

from stable_baselines3.common.vec_env import DummyVecEnv, SubprocVecEnv
from stable_baselines3 import A2C, PPO, DQN
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common.env_util import make_vec_env
from stable_baselines3.common.utils import set_random_seed

import tensorflow as tf

import numpy as np
import pandas as pd
import matplotlib
from matplotlib import pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
import seaborn as sns
import os
import datetime

import multiprocessing

import quantstats as qs

## Get values from the config file

In [2]:
# Parameters
#list_asset_ticket = ["BTCUSDT", "ETHUSDT", "BNBUSDT"]
list_asset_ticket = ["BTCUSDT"]
list_timestamp = ["1d", "1h", "15m", "5m"]  
list_feat = [1, 2, 3]

list_config_file_name = []
for asset_ticket in list_asset_ticket:
    for timestamp in list_timestamp:
        for feat in list_feat:
            config_file_name_temp = "config_msc_" + asset_ticket + "_" + timestamp + "_feat_package" + str(feat) + ".ini"
            list_config_file_name.append(config_file_name_temp)

In [3]:
list_config_file_name

['config_msc_BTCUSDT_1d_feat_package1.ini',
 'config_msc_BTCUSDT_1d_feat_package2.ini',
 'config_msc_BTCUSDT_1d_feat_package3.ini',
 'config_msc_BTCUSDT_1h_feat_package1.ini',
 'config_msc_BTCUSDT_1h_feat_package2.ini',
 'config_msc_BTCUSDT_1h_feat_package3.ini',
 'config_msc_BTCUSDT_15m_feat_package1.ini',
 'config_msc_BTCUSDT_15m_feat_package2.ini',
 'config_msc_BTCUSDT_15m_feat_package3.ini',
 'config_msc_BTCUSDT_5m_feat_package1.ini',
 'config_msc_BTCUSDT_5m_feat_package2.ini',
 'config_msc_BTCUSDT_5m_feat_package3.ini']

In [4]:
def get_and_process_data(ccy, data_frequency):
    try:
        
        # Get the data
        filename_data = 'binance_' + ccy + "_" + data_frequency + '_from_2017_01_01_to_2022_12_31_candlesticks_signals_processed_technical_indicators_and_crypto_index_log.csv'
        fullpath_data = os.path.join('../../data/50_log', filename_data)
        df = pd.read_csv(fullpath_data)

        # Converting Date Column to DateTime Type
        # Set the index on the dataframe
        df['date_index'] = df['date']
        df.set_index('date_index', inplace=True)

        return df

    except Exception as e:
        print(e)

In [5]:
def run():
    try:
        for element in list_config_file_name:
            
            # Get the config file
            configur = ConfigParser()
            config_file_name = element
            print (configur.read(os.path.join('../config_files', config_file_name)))
            
            config_file_name_without_extension = config_file_name.replace(".ini", "")

            # Get all the relevant values
            ccy = configur.get('data', 'ccy')
            data_frequency_train = configur.get('data', 'data_frequency_train')
            data_frequency_val = configur.get('data', 'data_frequency_val')

            window_size = configur.getint('environment', 'window_size')
            start_date = configur.get('environment', 'start_date')
            mid_date = configur.get('environment', 'mid_date')
            end_date = configur.get('environment', 'end_date')
            list_features = configur.get('environment', 'features').split(',')
            target = configur.get('environment', 'target')

            num_of_simulations = configur.getint('simulation', 'num_of_simulations')

            def my_process_data(env):
                start = env.frame_bound[0] - env.window_size
                end = env.frame_bound[1]
                prices = env.df.loc[:, target].to_numpy()[start:end]
                signal_features = env.df.loc[:, list_features].to_numpy()[start:end]
                dates = env.df.index.to_numpy()[start:end]
                return prices, signal_features, dates

            class MyEnv(CryptoEnvLogBLSH):
                _process_data = my_process_data

            df_train = get_and_process_data(ccy, data_frequency_train)
            df_val = get_and_process_data(ccy, data_frequency_val)

            start_date_id_train = int(df_train.index.get_loc(start_date))
            mid_date_id_train = int(df_train.index.get_loc(mid_date))
            mid_date_id_val = int(df_val.index.get_loc(mid_date))
            end_date_id_val = int(df_val.index.get_loc(end_date))

            def make_env(df, frame_bound, window_size, env_id: str, rank: int, seed: int = 0) -> Callable:
                """
                Utility function for multiprocessed env.
                
                :param env_id: (str) the environment ID
                :param num_env: (int) the number of environment you wish to have in subprocesses
                :param seed: (int) the inital seed for RNG
                :param rank: (int) index of the subprocess
                :return: (Callable)
                """
                def _init() -> gym.Env:
                    env = MyEnv(df=df, frame_bound=frame_bound, window_size=window_size)
                    env.seed(seed + rank)
                    return env
                set_random_seed(seed)
                return _init 

            list_models = ['A2C', 'PPO', 'DQN', 'RANDOM']

            path_tensorboard = os.path.join("tensorboard", config_file_name.replace(".ini", ""))

            df_sim_results = pd.DataFrame()

            num_cpu = 8  # Number of processes to use
            env_id = 'CryptoEnvLogBLSH-v1'

            #setting up our environment for training 
            env = SubprocVecEnv([make_env(df_train, (start_date_id_train, mid_date_id_train), window_size, env_id, i) for i in range(num_cpu)])

            sim_id = 1

            for model_name in list_models:

                if(model_name == 'A2C'):
                    model = A2C('MlpPolicy', env, verbose=0, tensorboard_log=path_tensorboard) 
                elif(model_name == 'PPO'):
                    model = PPO('MlpPolicy', env, batch_size=1024, verbose=0, tensorboard_log=path_tensorboard)
                elif(model_name == 'DQN'):
                    model = DQN('MlpPolicy', env, batch_size=1024, verbose=0, tensorboard_log=path_tensorboard)

                #setting the learning timesteps
                model.learn(total_timesteps=(mid_date_id_train - start_date_id_train))
                
                # Export the model components
                
                if(model_name == 'A2C'):
                    model.save(os.path.join('../save_models_components', 'model', 'model_' + model_name + "_" + config_file_name_without_extension))
                    model.policy.save(os.path.join('../save_models_components', 'policy', 'policy_' + model_name + "_" + config_file_name_without_extension))
                elif(model_name == 'PPO'):
                    model.save(os.path.join('../save_models_components', 'model', 'model_' + model_name + "_" + config_file_name_without_extension))
                    model.policy.save(os.path.join('../save_models_components', 'policy', 'policy_' + model_name + "_" + config_file_name_without_extension))
                elif(model_name == 'DQN'):
                    model.save(os.path.join('../save_models_components', 'model', 'model_' + model_name + "_" + config_file_name_without_extension))
                    model.save_replay_buffer(os.path.join('../save_models_components', 'replay_buffer', 'replay_buffer_' + model_name + "_" + config_file_name_without_extension))
                    model.policy.save(os.path.join('../save_models_components', 'policy', 'policy_' + model_name + "_" + config_file_name_without_extension))

                df_sim_results_temp = pd.DataFrame(columns=['total_reward_cash', 'total_profit_percentage', 'fmt_total_profit_percentage', 'num_of_trades'])

                list_sim_id = []

                for i in range(num_of_simulations):   
                    
                    list_sim_id.append(sim_id)

                    # # Check the progress
                    # if(i % 100 == 0):
                    #     print(f'model_name:{model_name} - sim_id:{sim_id}')

                    env = MyEnv(df=df_val, frame_bound=(mid_date_id_val,end_date_id_val), window_size=window_size)

                    #Setting up the Agent Environment
                    obs = env.reset()
                    
                    while True: 
                        obs = obs[np.newaxis, ...]

                        if(model_name == 'RANDOM'):
                            action = env.action_space.sample()
                        else:
                            action, _states = model.predict(obs)
                        
                        obs, rewards, done, info = env.step(action)
                        
                        if done:
                            df_sim_results_temp = df_sim_results_temp.append(info, ignore_index=True, sort=False)
                            break

                    # Export robot actions plot and data
                    fig, df_robot_actions = env.render_all()

                    # Export the history of details (the info dict inside of the environment)
                    df_sim_env_data = pd.DataFrame.from_dict(env.history)

                    # Merge robot actions data + sim env data
                    df_sim_env_data_initial = pd.DataFrame(columns=['total_reward_cash', 'total_profit_percentage', 'fmt_total_profit_percentage', 'num_of_trades'])
                    for i in range(0, window_size + 1):
                        df_sim_env_data_initial = df_sim_env_data_initial.append(pd.Series([0, 0, 0, 0], index=df_sim_env_data_initial.columns), ignore_index=True)
                    df_sim_env_data_temp = df_sim_env_data_initial.append(df_sim_env_data, ignore_index=True)
                    df_robot_actions_and_env = pd.merge(df_robot_actions, df_sim_env_data_temp, left_index=True, right_index=True, how = "outer")
                    
                    # Export the Robot actions & env data
                    file_export_robot_actions_and_env_results = config_file_name.replace(".ini", ".csv")
                    fullpath_export_robot_actions_and_env_results = os.path.join('../results', 'data', 'robot_actions_and_env', 'robot_actions_and_env_results_' + config_file_name_without_extension + '_sim_id_' + str(sim_id) + ".csv")        
                    df_robot_actions_and_env.to_csv(fullpath_export_robot_actions_and_env_results, index=False)

                    sim_id = sim_id + 1
                    
                df_sim_results_temp['sim_id'] = list_sim_id
                df_sim_results_temp['ccy'] = ccy
                df_sim_results_temp['data_frequency_train'] = data_frequency_train
                df_sim_results_temp['data_frequency_val'] = data_frequency_val
                df_sim_results_temp['window_size'] = window_size
                df_sim_results_temp['start_date'] = start_date
                df_sim_results_temp['mid_date'] = mid_date
                df_sim_results_temp['end_date'] = end_date
                df_sim_results_temp['list_features'] = str(list_features)
                df_sim_results_temp['model_type'] = model_name 
                df_sim_results_temp['num_of_simulations'] = num_of_simulations   

                df_sim_results = pd.concat([df_sim_results, df_sim_results_temp], axis=0)

            # Pre-process the results
            df_sim_results = df_sim_results.reset_index(drop=True)
            df_sim_results = df_sim_results.drop_duplicates()

            # Generate plots
            str_title = f"ccy:{ccy} | data_frequency_train:{data_frequency_train} | data_frequency_val:{data_frequency_val} | window_size:{window_size}<br>start_run_date:{mid_date} | end_run_date:{end_date} | lenght list_features:{len(list_features)} | num_of_simulations:{num_of_simulations}"
            file_export_plot_sim_results = config_file_name.replace(".ini", "")
            
            fig = px.box(df_sim_results, y="fmt_total_profit_percentage", color="model_type", points="all", color_discrete_sequence=[ "#FF7F0E", "#00CC96", "#10aded", "#8A56EF"],  width=800, height=600)
            fig.update_layout(title=str_title, font={'size': 8})
            fig.write_html(os.path.join('../results', 'plots', 'total_profit_percentage', 'box_plot_' + file_export_plot_sim_results + '.html'))

            fig = px.histogram(df_sim_results, x="fmt_total_profit_percentage", color="model_type", color_discrete_sequence=[ "#FF7F0E", "#00CC96", "#10aded", "#8A56EF"],  width=800, height=600, marginal="rug", # can be `box`, `violin`
                                    hover_data=['total_reward_cash','fmt_total_profit_percentage', 'num_of_trades', 'sim_id'])
            fig.update_layout(title=str_title, font={'size': 8})
            fig.write_html(os.path.join('../results', 'plots', 'total_profit_percentage', 'hist_plot_' + file_export_plot_sim_results + '.html'))
            
            fig = px.histogram(df_sim_results, x="num_of_trades", color="model_type", color_discrete_sequence=[ "#FF7F0E", "#00CC96", "#10aded", "#8A56EF"],  width=800, height=600, marginal="rug", # can be `box`, `violin`
                                    hover_data=['total_reward_cash','fmt_total_profit_percentage', 'num_of_trades', 'sim_id'])
            fig.update_layout(title=str_title, font={'size': 8})
            fig.write_html(os.path.join('../results', 'plots', 'num_of_trades', 'hist_plot_' + file_export_plot_sim_results + '.html'))  

            # Export the results
            file_export_sim_results = config_file_name.replace(".ini", ".csv")
            fullpath_export_sim_results = os.path.join('../results', 'data', 'simul_results', 'sim_results_' + file_export_sim_results)
            df_sim_results.to_csv(fullpath_export_sim_results, index=False)
            
    except Exception as e:
        print(e)

In [6]:
run()

['../config_files/config_msc_BTCUSDT_1d_feat_package1.ini']
['../config_files/config_msc_BTCUSDT_1d_feat_package2.ini']


KeyboardInterrupt: 